# Stats Helper
Given a aoa_evaluate result, print the fine-grained stats 

In [1]:
import torch
import numpy as np
import json

def geometric_mean(x): 
    a = np.log(x)
    return np.exp(a.mean())

def mean(x):
    if len(x) == 0:
        return 0
    return sum(x)/len(x)
with open("eval_mapping/unseen.json") as f:
    unseen_id_to_test_id_mapping = json.load(f)
unseen_test_id_to_result_id_mapping = {v: int(k) for k, v in unseen_id_to_test_id_mapping.items()}

with open("eval_mapping/seen.json") as f:
    seen_id_to_test_id_mapping = json.load(f)
seen_test_id_to_result_id_mapping = {v: int(k) for k, v in seen_id_to_test_id_mapping.items()}


In [2]:
unseen_word_list= ['pizza','crosswalk','cheese','star','classroom','fluffy','circular','sink','doctor','foreign','diverse','product','stove','steep','warm','donkey','aged','barefoot','elephant','heart','gym','security','cafe','teacher','button','student','newborn','pan','telephone','bamboo','brush']
with open("/scratch/chaijy_root/chaijy2/jiayipan/ACL/RefCloze/refcloze_data/0120_Unseen_All.json") as f:
    unseen_dataset = json.load(f)
with open("/scratch/chaijy_root/chaijy2/jiayipan/ACL/RefCloze/refcloze_data/flickr_seen_refcloze_dataset.json") as f:
    seen_dataset = json.load(f)
seen_word_list = list(set([t['mask_regions'][0][1] for t in seen_dataset['tests']]))


In [3]:
from collections import defaultdict
unseen_word_to_result_idx = defaultdict(list)
for i, test in enumerate(unseen_dataset['tests']):
    assert len(test['mask_regions']) == 1
    assert test['mask_regions'][0][1] in unseen_word_list
    unseen_word_to_result_idx[test['mask_regions'][0][1]].append(unseen_test_id_to_result_id_mapping[i])

seen_word_to_result_idx = defaultdict(list)
for i, test in enumerate(seen_dataset['tests']):
    assert len(test['mask_regions']) == 1
    assert test['mask_regions'][0][1] in seen_word_list
    seen_word_to_result_idx[test['mask_regions'][0][1]].append(seen_test_id_to_result_id_mapping[i])

In [4]:
print(seen_word_list)

['female', 'man', 'player', 'bench', 'top', 'yellow', 'little', 'guy', 'gray', 'floor', 'field', 'long', 'face', 'orange', 'woman', 'table', 'dress', 'snow', 'white', 'red', 'person', 'striped', 'food', 'child', 'water', 'bike', 'black', 'jeans', 'lady', 'street', 'hair', 'sign', 'large', 'dark', 'car', 'sidewalk', 'young', 'brown', 'baby', 'glasses', 'wall', 'dog', 'hand', 'blond', 'horse', 'girl', 'green', 'hat', 'shirt', 'jacket', 'guitar', 'boy', 'head', 'blue', 'building', 'small', 'pants', 'grass', 'beach', 'ball']


In [5]:
print(unseen_word_list)

['pizza', 'crosswalk', 'cheese', 'star', 'classroom', 'fluffy', 'circular', 'sink', 'doctor', 'foreign', 'diverse', 'product', 'stove', 'steep', 'warm', 'donkey', 'aged', 'barefoot', 'elephant', 'heart', 'gym', 'security', 'cafe', 'teacher', 'button', 'student', 'newborn', 'pan', 'telephone', 'bamboo', 'brush']


In [7]:
stats = torch.load("/scratch/chaijy_root/chaijy2/jiayipan/ACL/RefCloze/continue/origin/seen/15w.pth", map_location='cpu')

In [9]:
import numpy as np
def print_stats(stats, idx_list=None):
    if idx_list is None:
        idx_list = range(len(stats['raw']))
    selected_ppls = [stats['raw'][i]['ppl'] for i in idx_list]
    ppl = geometric_mean(selected_ppls)
    selected_all_ious = [stats['raw'][i]['all_iou'] for i in idx_list]
    aiou = mean(selected_all_ious)
    any_iou = mean([mean(stats['raw'][i]['any_iou']) for i in idx_list])
    gppl = ppl / aiou
    if ppl < 1e4:
        print(f"\t\t\tppl: {ppl:.2f}\t aiou: {aiou:.2f}\t gppl: {gppl:.2f}")
    # log_ppl = np.log(ppl)
    # log_gppl = np.log(gppl)
    ghr1 = 0
    return {"ppl": ppl, "aiou": aiou, "any_iou": any_iou, "gppl": gppl}

In [10]:
def print_seen_unseen(root_name, ckpt_name):
    Stats = {}
    try:
        stats_seen = torch.load(f"/scratch/chaijy_root/chaijy2/jiayipan/ACL/RefCloze/continue/{root_name}/seen/{ckpt_name}.pth", map_location="cpu")
    except:
        pass
    else:
        print("\tSeen")
        Stats['seen'] = {}
        Stats['seen']['all'] = print_stats(stats_seen)
        Stats['seen']['per_word'] = {}
        for w in seen_word_list:
            print(f"\t\t{w}")
            Stats['seen']['per_word'][w] = print_stats(stats_seen, seen_word_to_result_idx[w])
    try:
        stats_unseen = torch.load(f"/scratch/chaijy_root/chaijy2/jiayipan/ACL/RefCloze/continue/{root_name}/unseen/{ckpt_name}.pth", map_location="cpu")
    except:
        pass
    else:
        print("\tUnseen")
        Stats['unseen']= {}
        Stats['unseen']['all'] = print_stats(stats_unseen)
        Stats['unseen']['per_word'] = {}
        for w in unseen_word_list:
            print(f"\t\t{w}")
            Stats['unseen']['per_word'][w] = print_stats(stats_unseen, unseen_word_to_result_idx[w])
    return Stats

In [12]:
all_stats = {}

In [13]:
all_stats['all'] = {}
ckpt_names = ["ALL", "ALL_gl"]
for ckpt in ckpt_names:
    print(ckpt)
    all_stats['all'][ckpt] = print_seen_unseen("all", ckpt)
    print()

ALL
	Seen
			ppl: 11.68	 aiou: 0.50	 gppl: 23.25
		female
			ppl: 32.41	 aiou: 0.64	 gppl: 50.86
		man
			ppl: 1.60	 aiou: 0.65	 gppl: 2.47
		player
			ppl: 10.71	 aiou: 0.56	 gppl: 19.03
		bench
			ppl: 2.73	 aiou: 0.51	 gppl: 5.31
		top
			ppl: 9.44	 aiou: 0.41	 gppl: 23.14
		yellow
			ppl: 23.39	 aiou: 0.44	 gppl: 52.92
		little
			ppl: 3.04	 aiou: 0.64	 gppl: 4.77
		guy
			ppl: 803.58	 aiou: 0.58	 gppl: 1383.39
		gray
			ppl: 219.07	 aiou: 0.52	 gppl: 418.70
		floor
			ppl: 7.19	 aiou: 0.58	 gppl: 12.49
		field
			ppl: 4.99	 aiou: 0.67	 gppl: 7.41
		long
			ppl: 22.94	 aiou: 0.48	 gppl: 47.77
		face
			ppl: 7.10	 aiou: 0.34	 gppl: 20.92
		orange
			ppl: 10.16	 aiou: 0.47	 gppl: 21.53
		woman
			ppl: 8.01	 aiou: 0.58	 gppl: 13.84
		table
			ppl: 3.46	 aiou: 0.44	 gppl: 7.95
		dress
			ppl: 471.16	 aiou: 0.55	 gppl: 850.69
		snow
			ppl: 2.65	 aiou: 0.60	 gppl: 4.41
		white
			ppl: 6.06	 aiou: 0.50	 gppl: 12.12
		red
			ppl: 3.02	 aiou: 0.45	 gppl: 6.70
		person
			ppl: 128.23	 aiou:

In [14]:
all_stats['origin'] = {}
ckpt_name = ["15w", "15w_gl_probed"]
for ckpt in ckpt_name:
    print(ckpt)
    all_stats['origin'][ckpt] =  print_seen_unseen("origin", ckpt)
    print()

15w
	Seen
			ppl: 3.52	 aiou: 0.59	 gppl: 5.99
		female
			ppl: 5.35	 aiou: 0.64	 gppl: 8.33
		man
			ppl: 1.70	 aiou: 0.71	 gppl: 2.39
		player
			ppl: 1.24	 aiou: 0.58	 gppl: 2.12
		bench
			ppl: 2.13	 aiou: 0.58	 gppl: 3.70
		top
			ppl: 4.16	 aiou: 0.52	 gppl: 8.07
		yellow
			ppl: 3.20	 aiou: 0.59	 gppl: 5.39
		little
			ppl: 2.78	 aiou: 0.72	 gppl: 3.86
		guy
			ppl: 19.12	 aiou: 0.67	 gppl: 28.58
		gray
			ppl: 13.78	 aiou: 0.60	 gppl: 22.95
		floor
			ppl: 3.94	 aiou: 0.67	 gppl: 5.92
		field
			ppl: 2.04	 aiou: 0.77	 gppl: 2.63
		long
			ppl: 6.37	 aiou: 0.58	 gppl: 10.94
		face
			ppl: 3.65	 aiou: 0.38	 gppl: 9.62
		orange
			ppl: 3.14	 aiou: 0.53	 gppl: 5.93
		woman
			ppl: 2.46	 aiou: 0.69	 gppl: 3.59
		table
			ppl: 2.07	 aiou: 0.52	 gppl: 3.98
		dress
			ppl: 4.95	 aiou: 0.64	 gppl: 7.69
		snow
			ppl: 2.09	 aiou: 0.70	 gppl: 2.97
		white
			ppl: 2.88	 aiou: 0.62	 gppl: 4.65
		red
			ppl: 2.18	 aiou: 0.56	 gppl: 3.91
		person
			ppl: 7.81	 aiou: 0.63	 gppl: 12.44
		stripe

In [9]:
print_seen_unseen("stream", f"aged_gl")

{}

In [10]:
all_stats['stream'] = {}
for ckpt in unseen_word_list:
    print(ckpt)
    all_stats['stream'][ckpt] = print_seen_unseen("stream", ckpt)
    print()
    print(f"{ckpt}_gl")
    all_stats['stream'][f"{ckpt}_gl"] = print_seen_unseen("stream", f"{ckpt}_gl")
    print()

pizza

pizza_gl

crosswalk

crosswalk_gl

cheese

cheese_gl

star

star_gl

classroom

classroom_gl

fluffy

fluffy_gl

circular

circular_gl

sink

sink_gl

doctor

doctor_gl

foreign

foreign_gl

diverse

diverse_gl

product

product_gl

stove

stove_gl

steep

steep_gl

warm

warm_gl

donkey

donkey_gl

aged

aged_gl

barefoot

barefoot_gl

elephant

elephant_gl

heart

heart_gl

gym

gym_gl

security

security_gl

cafe

cafe_gl

teacher

teacher_gl

button

button_gl

student

student_gl

newborn

newborn_gl

pan

pan_gl

telephone

telephone_gl

bamboo

bamboo_gl

brush

brush_gl



In [11]:
all_stats['per_class'] = {}
for ckpt in unseen_word_list:
    print(ckpt)
    all_stats['per_class'][ckpt] = print_seen_unseen("per_class", ckpt)
    print()
    print(f"{ckpt}_gl")
    all_stats['per_class'][f"{ckpt}_gl"] = print_seen_unseen("per_class", f"{ckpt}_gl")
    print()

pizza
	Seen
			ppl: 6.66	 aiou: 0.55	 gppl: 12.00
		shirt
			ppl: 1.31	 aiou: 0.62	 gppl: 2.10
		orange
			ppl: 9.60	 aiou: 0.49	 gppl: 19.48
		hair
			ppl: 2.65	 aiou: 0.44	 gppl: 6.04
		pants
			ppl: 3.23	 aiou: 0.43	 gppl: 7.55
		ball
			ppl: 2.10	 aiou: 0.30	 gppl: 7.12
		green
			ppl: 6.52	 aiou: 0.51	 gppl: 12.81
		large
			ppl: 6.78	 aiou: 0.53	 gppl: 12.68
		face
			ppl: 7.87	 aiou: 0.35	 gppl: 22.28
		yellow
			ppl: 6.76	 aiou: 0.52	 gppl: 12.88
		female
			ppl: 15.16	 aiou: 0.66	 gppl: 22.97
		long
			ppl: 11.24	 aiou: 0.54	 gppl: 20.94
		blue
			ppl: 3.28	 aiou: 0.59	 gppl: 5.56
		dog
			ppl: 1.42	 aiou: 0.69	 gppl: 2.06
		glasses
			ppl: 4.91	 aiou: 0.25	 gppl: 19.70
		black
			ppl: 3.18	 aiou: 0.60	 gppl: 5.34
		jacket
			ppl: 5.59	 aiou: 0.55	 gppl: 10.09
		lady
			ppl: 50.52	 aiou: 0.67	 gppl: 74.98
		red
			ppl: 2.69	 aiou: 0.48	 gppl: 5.67
		brown
			ppl: 6.71	 aiou: 0.64	 gppl: 10.41
		building
			ppl: 15.91	 aiou: 0.63	 gppl: 25.33
		boy
			ppl: 2.16	 aiou: 0.71	 gpp

In [12]:
all_stats['per_class']['pizza_gl']

{}

In [13]:
with open("all_stats_8.json", 'w') as f:
    json.dump(all_stats, f, indent=4)

In [14]:
all_stats['all']["ALL"]['unseen']

{'all': {'ppl': 10.887543371579058,
  'aiou': 0.49626435436729494,
  'any_iou': 0.4034534947690128,
  'gppl': 21.938999397730218},
 'per_word': {'pizza': {'ppl': 2.947627612838465,
   'aiou': 0.37944436017423866,
   'any_iou': 0.38173056858523735,
   'gppl': 7.7682736185219134},
  'crosswalk': {'ppl': 545.5378293723969,
   'aiou': 0.42168316811323164,
   'any_iou': 0.3741357405201444,
   'gppl': 1293.7149751870281},
  'cheese': {'ppl': 4.733703466664192,
   'aiou': 0.31520108707249167,
   'any_iou': 0.21173367792193498,
   'gppl': 15.018042959907397},
  'star': {'ppl': 28.515778226410323,
   'aiou': 0.5103762258216739,
   'any_iou': 0.2874121798094303,
   'gppl': 55.872073940163844},
  'classroom': {'ppl': 3.3216091136214954,
   'aiou': 0.5902752780914307,
   'any_iou': 0.36029015262012853,
   'gppl': 5.627220445961139},
  'fluffy': {'ppl': 11.810403772346344,
   'aiou': 0.4841418319940567,
   'any_iou': 0.4645526074076242,
   'gppl': 24.394512086886408},
  'circular': {'ppl': 47.75556

In [15]:
all_stats['stream'].keys()

dict_keys(['pizza', 'pizza_gl', 'crosswalk', 'crosswalk_gl', 'cheese', 'cheese_gl', 'star', 'star_gl', 'classroom', 'classroom_gl', 'fluffy', 'fluffy_gl', 'circular', 'circular_gl', 'sink', 'sink_gl', 'doctor', 'doctor_gl', 'foreign', 'foreign_gl', 'diverse', 'diverse_gl', 'product', 'product_gl', 'stove', 'stove_gl', 'steep', 'steep_gl', 'warm', 'warm_gl', 'donkey', 'donkey_gl', 'aged', 'aged_gl', 'barefoot', 'barefoot_gl', 'elephant', 'elephant_gl', 'heart', 'heart_gl', 'gym', 'gym_gl', 'security', 'security_gl', 'cafe', 'cafe_gl', 'teacher', 'teacher_gl', 'button', 'button_gl', 'student', 'student_gl', 'newborn', 'newborn_gl', 'pan', 'pan_gl', 'telephone', 'telephone_gl', 'bamboo', 'bamboo_gl', 'brush', 'brush_gl'])

In [16]:
all_stats['stream']['pizza']['seen']['all']

KeyError: 'seen'

In [29]:
all_stats['stream']['pizza']['unseen']['all']

{'ppl': 27571.62510020724,
 'aiou': 0.48477719226587473,
 'gppl': 56874.839699730874}

In [31]:
all_stats['stream']['pizza']['unseen']['per_word']['pizza']

{'ppl': 1.0153479424194118,
 'aiou': 0.4146950714662671,
 'gppl': 2.4484205679835385}

In [33]:
all_stats['stream'][unseen_word_list[-1]]['seen']['per_word']

{'water': {'ppl': 4.113276265189779,
  'aiou': 0.4806632087100297,
  'gppl': 8.557501782232725},
 'top': {'ppl': 52.266270817325534,
  'aiou': 0.31150117233628405,
  'gppl': 167.78835991314017},
 'street': {'ppl': 4.0623143499105945,
  'aiou': 0.5006654018186965,
  'gppl': 8.113830784300252},
 'beach': {'ppl': 4.788661548565623,
  'aiou': 0.6552359709516168,
  'gppl': 7.3083007662276565},
 'car': {'ppl': 157.94442067119172,
  'aiou': 0.41005294674541803,
  'gppl': 385.18055271835846},
 'long': {'ppl': 83.11092861264463,
  'aiou': 0.3915649720234796,
  'gppl': 212.25322628618835},
 'glasses': {'ppl': 22.949227357872648,
  'aiou': 0.06539442809298635,
  'gppl': 350.93551587056373},
 'bench': {'ppl': 3.1011462524175264,
  'aiou': 0.4822570472024381,
  'gppl': 6.430484054939588},
 'dress': {'ppl': 351.8570354165248,
  'aiou': 0.44192228839965536,
  'gppl': 796.1966269923017},
 'table': {'ppl': 12.924983827213254,
  'aiou': 0.3958651446737349,
  'gppl': 32.64996679075092},
 'brown': {'ppl':